In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
import math

# sample 데이터 셋팅

In [3]:
# 우리강의 샘플 데이터
raw_sample = pd.DataFrame([[0,1,3.5],[0,2,4.5],[0,3,5],[0,4,4], [0,6,1]
                           ,[1,0,4.5],[1,2,5],[1,3,5], [1,5,4], [1,6,0.5]
                           ,[2,1,4],[2,2,2],[2,3,1], [2,5,4.5]
                           ,[3,1,3.5],[3,4,3], [3,5,4], [3,6,2]
                           , [4,3,5],[4,4,4.5]], columns=['userId', 'movieId', 'rating'])

raw_sample

,userId,movieId,rating
0,0,1,3.5
1,0,2,4.5
2,0,3,5.0
3,0,4,4.0
4,0,6,1.0
5,1,0,4.5
6,1,2,5.0
7,1,3,5.0
8,1,5,4.0
9,1,6,0.5


In [4]:
# raw_sample = raw_sample[raw_sample['movieId'].isin([0,1,2,3,4,5,6])]
# raw_sample

In [5]:
ui_metrix = raw_sample.set_index(['movieId', 'userId']).unstack()['rating']
ui_metrix

userId,0,1,2,3,4
movieId,,,,,
0,NaN,4.5,NaN,NaN,NaN
1,3.5,NaN,4.0,3.5,NaN
2,4.5,5.0,2.0,NaN,NaN
3,5.0,5.0,1.0,NaN,5.0
4,4.0,NaN,NaN,3.0,4.5
5,NaN,4.0,4.5,4.0,NaN
6,1.0,0.5,NaN,2.0,NaN


In [6]:
ui_metrix[4] - ui_metrix[4].mean()

movieId
0     NaN
1     NaN
2     NaN
3    0.25
4   -0.25
5     NaN
6     NaN
Name: 4, dtype: float64

# 코사인 유사도

## 1. 위키피디아 ver

In [7]:
# 위키피디아 코사인 유사도 구현
def cosine_m(u, v):
    u_ratings = ui_metrix[u]
    v_ratings = ui_metrix[v]
    
    u_items = set(u_ratings[u_ratings.notna()].index)
    v_items = set(v_ratings[v_ratings.notna()].index)
    commons = list(u_items & v_items)
    
    uv_dot = np.dot(u_ratings[commons], v_ratings[commons])
    ab_length_multiply = math.sqrt(sum(i**2 for i in u_ratings.fillna(0)))*math.sqrt(sum(i**2 for i in v_ratings.fillna(0)))
    
    
#     print(u_ratings[u_ratings.notna()].index)
#     print(u_items)
#     print(ui_metrix[u_ratings.notna()])
#     print(commons)
    print(uv_dot)
    print(ab_length_multiply)
    
    return uv_dot/ab_length_multiply

In [8]:
print(f'Cosine_sim(1,0) : {cosine_m(1, 0)}')
print(f'Cosine_sim(1,1) : {cosine_m(1, 1)}')
print(f'Cosine_sim(1,2) : {cosine_m(1, 2)}')
print(f'Cosine_sim(1,3) : {cosine_m(1, 3)}')
print(f'Cosine_sim(1,4) : {cosine_m(1, 4)}')

48.0
80.27608610289866
Cosine_sim(1,0) : 0.5979364756083542
86.5
86.5
Cosine_sim(1,1) : 1.0
33.0
59.733784410499226
Cosine_sim(1,2) : 0.5524511852994147
17.0
59.733784410499226
Cosine_sim(1,3) : 0.28459606515424396
25.0
62.562968279965745
Cosine_sim(1,4) : 0.3995974086160109


## 2. Surprise 패키지 ver

In [9]:
# surprise package 코사인 유사도 구현
def cosine_s(u, v):
    u_ratings = ui_metrix[u]
    v_ratings = ui_metrix[v]
    
    u_items = set(u_ratings[u_ratings.notna()].index)
    v_items = set(v_ratings[v_ratings.notna()].index)
    commons = list(u_items & v_items)
    
    uv_dot = np.dot(u_ratings[commons], v_ratings[commons])
    ab_length_multiply = math.sqrt(sum(i**2 for i in u_ratings[commons]))*math.sqrt(sum(i**2 for i in v_ratings[commons]))
    
    
#     print(u_ratings[u_ratings.notna()].index)
#     print(u_items)
#     print(ui_metrix[u_ratings.notna()])
#     print(commons)
#     print(u_ratings.iloc[list(commons)])
#     print(uv_dot/ab_length_multiply)

    print(uv_dot)
    print(ab_length_multiply)

    return uv_dot/ab_length_multiply


In [10]:
print(f'Cosine_sim(1,0) : {cosine_s(1, 0)}')
print(f'Cosine_sim(1,1) : {cosine_s(1, 1)}')
print(f'Cosine_sim(1,2) : {cosine_s(1, 2)}')
print(f'Cosine_sim(1,3) : {cosine_s(1, 3)}')
print(f'Cosine_sim(1,4) : {cosine_s(1, 4)}')

48.0
48.20853140264698
Cosine_sim(1,0) : 0.9956743879852865
86.5
86.5
Cosine_sim(1,1) : 1.0
33.0
40.82278775390039
Cosine_sim(1,2) : 0.808372034730701
17.0
18.027756377319946
Cosine_sim(1,3) : 0.9429903335828895
25.0
25.0
Cosine_sim(1,4) : 1.0


In [11]:
math.sqrt(50+0.25)*math.sqrt(4.5**2+25+1)

48.20853140264698

## 3. Surprise 패키지 확인

In [12]:
from surprise import Reader, Dataset

# 서프라이즈 패키지에게 rating의 최솟값, 최댓값 알려줌
reader = Reader(rating_scale=(0,5))
data = Dataset.load_from_df(raw_sample, reader)
trainset = data.build_full_trainset()

In [13]:
from surprise.prediction_algorithms.knns import KNNBasic

# 모델 셋팅
sim_options = {'name': 'cosine',
               'min_support' : 0,
              'user_based': True}

model = KNNBasic(k=2, min_k=1, sim_options=sim_options, verbose=True)

model.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [14]:
pd.DataFrame(model.sim)

,0,1,2,3,4
0,1.000000,0.995674,0.805776,0.965908,0.998314
1,0.995674,1.000000,0.808372,0.942990,1.000000
2,0.805776,0.808372,1.000000,0.999969,1.000000
3,0.965908,0.942990,0.999969,1.000000,1.000000
4,0.998314,1.000000,1.000000,1.000000,1.000000


# pearson 유사도

## 1. 위키피디아 ver

In [15]:
#  pearson 유사도 위키피디아 버전 구현
def pearson_m(u, v):
    u_ratings = ui_metrix[u]
    v_ratings = ui_metrix[v]
    
    u_mean = raw_sample[raw_sample['userId'] == u]['rating'].mean()
    v_mean = raw_sample[raw_sample['userId'] == v]['rating'].mean()
    
    u_items = set(u_ratings[u_ratings.notna()].index)
    v_items = set(v_ratings[v_ratings.notna()].index)
    commons = list(u_items & v_items)
    
    uv_dot = np.dot(u_ratings[commons] - u_mean, v_ratings[commons] - v_mean)
    ab_length_multiply = math.sqrt(sum(i**2 for i in (u_ratings[commons]-u_mean)))*math.sqrt(sum(i**2 for i in (v_ratings[commons]-v_mean)))
    
#     print(sum([i**2 for i in (u_ratings[commons]-u_mean)]))
#     print([i**2 for i in (v_ratings[commons]-v_mean)])
#     print(u_ratings[u_ratings.notna()].index)
#     print(u_items)
#     print(ui_metrix[u_ratings.notna()])
#     print(commons)
#     print(u_ratings[commons]-u_mean)
#     print(v_ratings[commons]-v_mean)
#     print(u_mean)
#     print(v_mean)
    print(f'분자 : {uv_dot}')
    print(f'분모 : {ab_length_multiply}')
    
    return uv_dot/ab_length_multiply if ab_length_multiply != 0 else 0

In [16]:
print(f'Pearson_sim(1,0) : {pearson_m(1, 0)}')
print(f'Pearson_sim(1,1) : {pearson_m(1, 1)}')
print(f'Pearson_sim(1,2) : {pearson_m(1, 2)}')
print(f'Pearson_sim(1,3) : {pearson_m(1, 3)}')
print(f'Pearson_sim(1,4) : {pearson_m(1, 4)}')

분자 : 11.34
분모 : 11.45548340315676
Pearson_sim(1,0) : 0.9899189410789128
분자 : 14.3
분모 : 14.300000000000002
Pearson_sim(1,1) : 0.9999999999999999
분자 : -2.9750000000000005
분모 : 4.495761893161158
Pearson_sim(1,2) : -0.6617343335120788
분자 : 3.8875
분모 : 4.711853403916552
Pearson_sim(1,3) : 0.8250468906287833
분자 : 0.30000000000000004
분모 : 0.30000000000000004
Pearson_sim(1,4) : 1.0


In [17]:
(1*(4.5-4.75) + 0.99*(4-3.6))/1.99 + 3.8

3.873366834170854

## 2. Surprise 패키지 ver

https://github.com/NicolasHug/Surprise/issues/224

In [18]:
def rating(u, i):
    # return r_ui
    return next(r for (j, r) in trainset.ur[u] if j == i)

def pearson_like_surprise(u, v):
    # computation like in surprise

    items_u = {i for (i,_ ) in trainset.ur[u]}
    items_v = {i for (i,_ ) in trainset.ur[v]}
    common = items_u & items_v
    n = len(common)

    sum_u = sum(rating(u, i) for i in common)
    sum_v = sum(rating(v, i) for i in common)

    num = n * sum((rating(u, i)) * (rating(v, i)) for i in common)

    num -= sum_u * sum_v
    a = n * sum(rating(u, i)**2 for i in common) - sum_u**2
    b = n * sum(rating(v, i)**2 for i in common) - sum_v**2
    denum = np.sqrt(a) * np.sqrt(b)
    return num / denum

# surprise package 제작자가 짠 코드 ver
def pearson_std_formula(u, v):
    # standard formula
    mu = {}
    for user, ratings in trainset.ur.items():
        mu[user] = np.mean([r for (_, r) in ratings])

    items_u = {i for (i,_ ) in trainset.ur[u]}
    items_v = {i for (i,_ ) in trainset.ur[v]}
    common = items_u & items_v

    num = sum((rating(u, i) - mu[u]) * (rating(v, i) - mu[v])
              for i in common)
    a = sum((rating(u, i) - mu[u])**2 for i in common)
    b = sum((rating(v, i) - mu[v])**2 for i in common)
    denum = np.sqrt(a) * np.sqrt(b)
    return num / denum

In [19]:
user_0 = trainset.to_inner_uid(1)
for other in range(0, 5):
    other = trainset.to_inner_uid(other)
    print(pearson_like_surprise(user_0, other)) # 패키지 제작자가 짠 코드
#     print(algo.sim[user_0, other]) # 패키지 연산 결과
    print(pearson_std_formula(user_0, other)) # 위키피디아 버전 피어슨 유사도
    print('-')

0.9933992677987827
0.9899189410789128
-
1.0
0.9999999999999999
-
-0.9607689228305226
-0.6617343335120788
-
1.0
0.8250468906287833
-
nan
1.0
-


## 3. Surprise 패키지 확인

In [20]:
from surprise import Reader, Dataset

# 서프라이즈 패키지에게 rating의 최솟값, 최댓값 알려줌
reader = Reader(rating_scale=(0,5))
data = Dataset.load_from_df(raw_sample, reader)
trainset = data.build_full_trainset()
anti_set = trainset.build_anti_testset()

In [21]:
# 모델 셋팅
sim_options = {'name': 'pearson_baseline',
              'min_support': 0,
              'user_based': True}

model = KNNBasic(k=2, min_k=0, sim_options=sim_options)

model.fit(data.build_full_trainset())

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [22]:
pd.DataFrame(model.sim)

,0,1,2,3,4
0,1.000000,0.019417,-0.019394,0.014896,0.009491
1,0.019417,1.000000,-0.016870,0.009440,0.000000
2,-0.019394,-0.016870,1.000000,0.008442,0.000000
3,0.014896,0.009440,0.008442,1.000000,0.000000
4,0.009491,0.000000,0.000000,0.000000,1.000000
